# Import Libraries

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, Activation, Lambda, dot, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

# Model

In [ ]:
directory = '../data/'

Mounted at /content/drive


In [ ]:
with open(directory + 'data_split.pkl', 'rb') as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

In [ ]:
def attention_3d_block(inputs):
    a = Dense(units=inputs.shape[-1], activation='softmax')(inputs)
    a_probs = Activation('softmax')(a)
    output_attention_mul = dot([inputs, a_probs], axes=1)
    return output_attention_mul

input_shape = X_train.shape[1]
inputs = Input(shape=(input_shape,))
attention_mul = attention_3d_block(inputs)
output = Dense(1)(attention_mul)
model = Model(inputs=[inputs], outputs=[output])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae', 'mse'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
98/98 [==============================] - 2s 8ms/step - loss: 0.9492 - mae: 0.5083 - mse: 0.9492 - val_loss: 0.4973 - val_mae: 0.4672 - val_mse: 0.4973
Epoch 2/50
98/98 [==============================] - 0s 3ms/step - loss: 0.9161 - mae: 0.4824 - mse: 0.9161 - val_loss: 0.4684 - val_mae: 0.4433 - val_mse: 0.4684
Epoch 3/50
98/98 [==============================] - 0s 4ms/step - loss: 0.8908 - mae: 0.4603 - mse: 0.8908 - val_loss: 0.4023 - val_mae: 0.4175 - val_mse: 0.4023
Epoch 4/50
98/98 [==============================] - 0s 3ms/step - loss: 0.8609 - mae: 0.4397 - mse: 0.8609 - val_loss: 0.3676 - val_mae: 0.3957 - val_mse: 0.3676
Epoch 5/50
98/98 [==============================] - 0s 3ms/step - loss: 0.8422 - mae: 0.4213 - mse: 0.8422 - val_loss: 0.3510 - val_mae: 0.3777 - val_mse: 0.3510
Epoch 6/50
98/98 [==============================] - 0s 3ms/step - loss: 0.8294 - mae: 0.4064 - mse: 0.8294 - val_loss: 0.3413 - val_mae: 0.3647 - val_mse: 0.3413
Epoch 7/50
98/98 [==========

In [ ]:
loss, mae, mse = model.evaluate(X_test, y_test)

y_pred = model.predict(X_test)

ss_res = np.sum((y_test - y_pred.flatten())**2)
ss_tot = np.sum((y_test - np.mean(y_test))**2)
r2 = 1 - (ss_res / ss_tot)

print(f'Loss: {loss}')
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

31/31 [==============================] - 0s 2ms/step
Loss: 0.656536877155304
Mean Squared Error: 0.656536877155304
Mean Absolute Error: 0.3045461177825928
R-squared: 0.3055974032443759


In [ ]:
print("Shape of pred: ", y_pred.shape)

# Export

In [ ]:
results = {
    'y_pred': y_pred,
    'mse': mse,
    'mae': mae,
    'r2': r2
}

with open(directory + 'attention_results.pkl', 'wb') as file:
    pickle.dump(results, file)